# trestles_visual notebook
### Author:   Brenton Blair, Updated:  August 19, 2019

### This notebook provides a visualization of low-level data elements extracted with trestles.  This includes the QP-delta, Macroblock Type, and DCT coefficients.  The variation of prediction footprint (VPF) score is also computed and plotted.  The distribution of the non-zero DCT coefficients is plotted on a per frame basis.  For more information on these data elements, please see the README provided with Trestles.

## Provide the following inputs in the first cell:
###  filename - input video file (e.g. mp4)
###  TAU -  tuning parameter used for VPF. Defined as the temporal window length (number of frames) used to compute standard deviation.
###   BETA - tuning parameter used for VPF. Defined as number of standard deviations of frame type counts in a given time window.

In [ ]:
filename = './test_vids/000.mp4'
TAU = 20
BETA = 3

In [ ]:
import os
import numpy
import scipy
import matplotlib
import subprocess
import shutil
import optparse
import sys
import IPython

### For brevity, functions called within this notebook are defined in a separate script

In [ ]:
%run tile_gen_functions

### Several sub-directories will be created in the current directory:

In [ ]:
!mkdir temp
%cd 'temp'
os.system("mkdir histograms_pngs qp_delta_pngs mb_pngs coeff_pngs vpf_pngs")

In [ ]:
quant = "1"
chroma = "0"
i_frames = "0"
details = "1"
motion_vector = "1"
output_path = "./temp/out"
trestles_exe = "../../ldecod.exe"
config_file = "../../decoder.cfg"
ffmpeg_exe = "ffmpeg"
ffprobe_exe = "ffprobe"

# ffprobe_exe = "ffprobe"
os.system("mkdir -p " + output_path)

In [ ]:
vid_dir       = os.path.dirname(filename)
vid_filename  = os.path.basename(filename)
vid_id        = os.path.splitext(vid_filename)[0]    # vid_filename.split(".")[0]

path_264_     = os.path.join(output_path, vid_id + '.264')
mb_path       = os.path.join(output_path, vid_id + '_mb_data.csv')
coef_path     = os.path.join(output_path, vid_id + '_coef_data.csv')
mb_info_path  = os.path.join(output_path, vid_id + '_mb_info.csv')
motion_path   = os.path.join(output_path, vid_id + '_mv_data.txt')
chroma_b_path = os.path.join(output_path, vid_id + '_chroma_b.txt')
chroma_r_path = os.path.join(output_path, vid_id + '_chroma_r.txt')

In [ ]:
# print (vid_dir)
# print (vid_filename)
# print (vid_id)

# print (path_264_)
# print (mb_path)
# print (coef_path)
# print (mb_info_path)
# print (motion_path)
# print (chroma_b_path)
# print (chroma_r_path)

In [ ]:
!pwd

### Use FFmpeg to extract the video stream from the mp4 container

In [ ]:
subprocess.check_call([ffmpeg_exe,"-y","-i",filename,"-vcodec","copy","-bsf","h264_mp4toannexb","-an","-f","h264",path_264_])

### Use Trestles to extract all the low-level data needed to create the visualizations

In [ ]:
subprocess.check_call([trestles_exe,"-d",config_file,"-p","InputFile=" + path_264_,"-p","OutputFile=" + "output.yuv","-p","DCT_quant="+quant,"-p","Chroma="+chroma,"-p","IFrames="+i_frames,"-p","Details="+details,"-p","MB_file=" + mb_path,"-p","RefFile = "" ","-p","CoefFile =" + coef_path,"-p","MB_info =" + mb_info_path,"-p","MV_flag=", motion_vector,"-p","MV_file =" + motion_path,"-p","CoefFile_b =" + chroma_b_path,"-p","CoefFile_r =" + chroma_r_path])

### FFprobe gets the number of frames, frame rate, resolution height and width

In [ ]:
status, out_frmnum = subprocess.getstatusoutput(ffprobe_exe + ' -v error -count_frames -select_streams v:0 -show_entries stream=nb_read_frames -of default=nokey=1:noprint_wrappers=1 %s' % (filename) )
status, out_height = subprocess.getstatusoutput('('+ ffprobe_exe +' -v error -of flat=s=_ -select_streams v:0 -show_entries stream=height %s)' % (filename))
status, out_width = subprocess.getstatusoutput('('+ ffprobe_exe +' -v error -of flat=s=_ -select_streams v:0 -show_entries stream=width %s)' % (filename))
status, out_frate = subprocess.getstatusoutput(ffprobe_exe + ' -v error -select_streams v -of default=noprint_wrappers=1:nokey=1  -show_entries stream=r_frame_rate %s'%(filename))

(frmnum,height,width,frate_str,frate) =  convert_ffprobe_out(out_frmnum,out_height,out_width,out_frate)

### Create QP-Delta video

In [ ]:
qpd_vid = os.path.join(output_path, "qpd_tile.mp4")
qp_type_video(ffmpeg_exe, mb_info_path, qpd_vid, frmnum, width, height, frate_str)

### Create Macroblock Type video

In [ ]:
mb_vid = os.path.join(output_path, "mb_tile.mp4")
mb_type_video(ffmpeg_exe, mb_info_path, mb_vid, frmnum, width, height, frate_str)

### Create DCT (integer-transform) coefficient map video

In [ ]:
coef_vid = os.path.join(output_path, "coeff_tile.mp4")
coef_type_video(ffmpeg_exe, coef_path, coef_vid, frmnum, width, height, frate_str)

### Create two offset videos (t+1, t+2)

In [ ]:
cut1_vid = os.path.join(output_path, "cut1.mp4")
cut2_vid = os.path.join(output_path, "cut2.mp4")
offset_frame(ffmpeg_exe, filename, 1, frate, cut1_vid)
offset_frame(ffmpeg_exe, filename, 2, frate, cut2_vid)

### Generate VPF video

In [ ]:
vpf_vid = os.path.join(output_path, "vpf_video.mp4")
vpf_video_gen(ffmpeg_exe, mb_path, TAU, BETA, vpf_vid, width, height, frate_str)

### Generate Histogram video

In [ ]:
hist_vid = os.path.join(output_path, "hist.mp4")
histogram_gen(ffmpeg_exe, coef_path, hist_vid, width, height, frate_str, frmnum)

## Generate Final 3x3 Video


In [ ]:
%cd '..'
%run tile_gen_functions

In [ ]:
top_vid = os.path.join(output_path, "top.mp4")
mid_vid = os.path.join(output_path, "middle.mp4")
btm_vid = os.path.join(output_path, "bottom.mp4")
output_vid = os.path.join(output_path, "output.mp4")
output_slow_vid = os.path.join(output_path, "output_slow.mp4")

tile_3_side_by_side(ffmpeg_exe, cut2_vid, cut1_vid, filename, top_vid)
tile_3_side_by_side(ffmpeg_exe, qpd_vid, mb_vid, coef_vid, mid_vid)
tile_side_by_side(ffmpeg_exe, vpf_vid, hist_vid, btm_vid)
tile_3_top_to_bottom(ffmpeg_exe, top_vid, mid_vid, btm_vid, output_vid)

In [ ]:
adjust_frame_rate(ffmpeg_exe, 10, output_vid, output_slow_vid)

## Display the Visualization Result

In [ ]:
import io
import base64
from IPython.display import HTML


video = io.open(output_slow_vid, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls width="750">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

In [ ]:
# from IPython.display import Video
# Video("trestles/output_slow.mp4")

<!-- Another approach... -->
<!-- <video controls src= "file:///Volumes/Dharma3/working/calaveras/trestles/output_slow.mp4" width="500"/> -->

Clean up: This is optional, if you run the following cell it will remove all sub-videos and it will be necessary to re-run the notebook to create them again.

In [ ]:
os.remove(qpd_vid)
os.remove(mb_vid)
os.remove(coef_vid)
os.remove(cut1_vid)
os.remove(cut2_vid)
os.remove(vpf_vid)
os.remove(hist_vid)
os.remove(top_vid)
os.remove(mid_vid)
os.remove(btm_vid)
os.remove(output_vid)
os.remove(output_slow_vid)